<a href="https://colab.research.google.com/github/prateekmanral011/hackathon_spg_slb/blob/main/files_to_csv_format.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Converting .dev files to csv files**

In [ ]:
# Colab-ready: robust parser for .dev files with irregular spacing
import re
import pandas as pd
from google.colab import files
from io import StringIO

# Upload the .dev file
print("Upload your .dev/.txt file")
uploaded = files.upload()
dev_file = list(uploaded.keys())[0]
print("Parsing:", dev_file)

def try_float(s):
    try:
        float(s)
        return True
    except:
        return False

# Read full content
with open(dev_file, "r", encoding="utf-8", errors="ignore") as f:
    lines = [ln.rstrip("\n\r") for ln in f]

# Drop blank lines and common comment lines (start with # or ~ or //)
lines = [ln for ln in lines if ln.strip() and not ln.strip().startswith(("#", "~", "//"))]

# If file is quoted CSV-like, try simple csv first
sample = "\n".join(lines[:30])
if "\t" in sample:
    primary_sep = "\t"
elif sample.count(",") >= sample.count(" "):
    primary_sep = ","
else:
    primary_sep = None  # fallback to regex whitespace split

rows = []
if primary_sep in (",", "\t"):
    # Try parsing by chosen separator but fallback to regex if row lengths vary
    for ln in lines:
        parts = [p.strip() for p in ln.split(primary_sep)]
        rows.append(parts)
    # check consistency
    lens = set(len(r) for r in rows)
    if len(lens) > 1:
        # inconsistent -> fallback to regex whitespace
        primary_sep = None

if primary_sep is None:
    # Split on any run of whitespace (handles variable spaces/tabs)
    for ln in lines:
        # If line contains commas inside quotes, remove surrounding quotes then split commas first
        if '"' in ln or "'" in ln:
            # remove surrounding quotes around delimited values to avoid splitting inside quotes
            # simple approach: replace commas inside quotes with a special token
            def _replace_inside_quotes(s, char=",", token="<<COMMA>>"):
                out = []
                in_q = False
                qchar = None
                for ch in s:
                    if ch in ('"', "'"):
                        if not in_q:
                            in_q = True
                            qchar = ch
                        elif ch == qchar:
                            in_q = False
                            qchar = None
                        out.append(ch)
                    elif in_q and ch == char:
                        out.append(token)
                    else:
                        out.append(ch)
                return "".join(out)
            ln2 = _replace_inside_quotes(ln, ",", "<<COMMA>>")
            parts = re.split(r'\s+', ln2.strip())
            # restore commas inside tokens
            parts = [p.replace("<<COMMA>>", ",") for p in parts]
        else:
            parts = re.split(r'\s+', ln.strip())
        rows.append(parts)

# Normalize row lengths by padding with empty strings / NaN
max_cols = max(len(r) for r in rows)
norm_rows = [r + [""]*(max_cols - len(r)) for r in rows]

# Detect header: if first row contains any non-numeric token (and >0 alphabetic tokens), treat it as header
first_row = norm_rows[0]
alpha_tokens = sum(1 for t in first_row if re.search(r'[A-Za-z]', t))
numeric_tokens = sum(1 for t in first_row if try_float(t))
has_header = alpha_tokens > numeric_tokens and alpha_tokens > 0

if has_header:
    header = [h.strip() if h.strip() else f"col{i+1}" for i,h in enumerate(first_row)]
    data_rows = norm_rows[1:]
else:
    # create generic headers col_1 ... col_n
    header = [f"col_{i+1}" for i in range(max_cols)]
    data_rows = norm_rows

# Build DataFrame
df = pd.DataFrame(data_rows, columns=header)

# Try convert numeric columns to numeric dtype
for col in df.columns:
    # replace common missing markers
    df[col] = df[col].replace(["", "-", "NA", "N/A", "null", "None"], pd.NA)
    df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='ignore')

# Save CSV
out_csv = dev_file.rsplit(".",1)[0] + "_converted.csv"
df.to_csv(out_csv, index=False)
print(f"Saved: {out_csv}")
print("\nDetected columns:", len(df.columns))
print(df.head(10))

# Offer download
files.download(out_csv)




Upload your .dev/.txt file


Saving A1.dev to A1.dev
Parsing: A1.dev
Saved: A1_converted.csv

Detected columns: 10
            MD            X            Y            Z          TVD  \
0  1713.982300  454755.6538  6783765.045 -1713.982300  1713.982300   
1  1963.229574  454563.1755  6783765.045 -1872.337324  1872.337324   
2  1977.535394  454552.0157  6783765.045 -1881.287427  1881.287427   
3  1991.841213  454540.6359  6783765.045 -1889.956128  1889.956128   
4  2006.147033  454529.0432  6783765.045 -1898.338021  1898.338021   
5  2020.452853  454517.2449  6783765.045 -1906.427882  1906.427882   
6  2034.758673  454505.2483  6783765.045 -1914.220668  1914.220668   
7  2049.064492  454493.0609  6783765.045 -1921.711520  1921.711520   
8  2063.370312  454480.6903  6783765.045 -1928.895770  1928.895770   
9  2077.676132  454468.1441  6783765.045 -1935.768938  1935.768938   

           DX   DY   AZIM       INCL  DLS  
0    0.000000  0.0  270.0  50.555312  0.0  
1 -192.478284  0.0  270.0  50.555312  0.0  
2 -203.6381

/tmp/ipython-input-2123604016.py:106: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='ignore')
/tmp/ipython-input-2123604016.py:106: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='ignore')
/tmp/ipython-input-2123604016.py:106: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='ignore')
/tmp/ipython-input-2123604016.py:106: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passin

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**converting las file to csv file**

In [ ]:
# Install dependency
!pip install lasio

import os
import lasio
import pandas as pd
from google.colab import files
import zipfile
from io import BytesIO

# Prompt user to upload multiple LAS files
print("Upload one or more .las files (select multiple using Ctrl/Cmd + click)...")
uploaded = files.upload()   # upload multiple files at once

if not uploaded:
    raise SystemExit("No files uploaded.")

out_csv_paths = []

def safe_read_las_to_df(path):
    """Try lasio.read, fallback to simple manual parse if needed."""
    try:
        las = lasio.read(path)
        df = las.df()
        # lasio sometimes returns empty DF; handle that
        if df is None or df.shape[0] == 0:
            raise ValueError("lasio returned empty dataframe")
        # Reset index so depth becomes a column
        df = df.reset_index()
        return df
    except Exception as e:
        print(f"lasio failed for {path}: {e}. Trying simple fallback parser...")
        # Simple fallback: collect lines after ~A and split whitespace
        data = []
        started = False
        with open(path, "r", errors="ignore") as f:
            for line in f:
                ln = line.rstrip("\n\r")
                if ln.strip().upper().startswith("~A"):  # data section
                    started = True
                    continue
                if not started:
                    continue
                if not ln.strip():
                    continue
                parts = ln.strip().split()
                data.append(parts)
        if not data:
            raise ValueError("Fallback parser found no data")
        # pad rows to equal length
        max_cols = max(len(r) for r in data)
        norm = [r + [""]*(max_cols - len(r)) for r in data]
        col_names = [f"col_{i+1}" for i in range(max_cols)]
        df = pd.DataFrame(norm, columns=col_names)
        # try convert numeric columns where possible
        for c in df.columns:
            df[c] = pd.to_numeric(df[c].astype(str).str.replace(',', ''), errors='ignore')
        return df

# Process each uploaded file
for fname in uploaded.keys():
    # Save uploaded bytes to disk (files.upload already did this in Colab)
    print(f"Processing: {fname}")
    root, ext = os.path.splitext(fname)
    ext_l = ext.lower()
    try:
        if ext_l == ".las" or ext_l == ".log":
            df = safe_read_las_to_df(fname)
        else:
            # If user uploaded a CSV-like .las with different ext, still try lasio
            try:
                df = safe_read_las_to_df(fname)
            except Exception:
                # As a fallback, try pandas with whitespace delimiter
                df = pd.read_csv(fname, sep=r"\s+", engine="python", header=None)
        out_name = f"{root}_converted.csv"
        df.to_csv(out_name, index=False)
        out_csv_paths.append(out_name)
        print(f" -> saved: {out_name} (rows: {len(df)}, cols: {df.shape[1]})")
    except Exception as e:
        print(f"ERROR converting {fname}: {e}")

if not out_csv_paths:
    raise SystemExit("No CSVs were generated.")

# Offer individual downloads
print("\nPreparing downloads...")
for p in out_csv_paths:
    print("Downloading:", p)
    files.download(p)

# Also create a ZIP with all CSVs for convenience
zip_name = "las_csvs_bundle.zip"
with zipfile.ZipFile(zip_name, "w", zipfile.ZIP_DEFLATED) as zf:
    for p in out_csv_paths:
        zf.write(p, arcname=os.path.basename(p))

print(f"\nCreated ZIP: {zip_name}")
files.download(zip_name)

print("Done.")


Upload one or more .las files (select multiple using Ctrl/Cmd + click)...


Saving A1.las to A1.las
Processing: A1.las
 -> saved: A1_converted.csv (rows: 2521, cols: 6)

Preparing downloads...
Downloading: A1_converted.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Created ZIP: las_csvs_bundle.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done.


**Converting fault stick depth files to Csv files **

In [ ]:
from google.colab import files
uploaded = files.upload()  # Select your ASCII file
filename = list(uploaded.keys())[0]
import pandas as pd
import re

def make_unique(colnames):
    counts = {}
    new_cols = []
    for col in colnames:
        if col in counts:
            counts[col] += 1
            new_cols.append(f"{col}_{counts[col]}")
        else:
            counts[col] = 0
            new_cols.append(col)
    return new_cols

# Read the file and find where the header ends
with open(filename, 'r') as f:
    lines = f.readlines()
    header_end = 0
    for i, line in enumerate(lines):
        if re.search(r'END HEADER', line, re.IGNORECASE):
            header_end = i + 1
            break
        if line.strip().startswith('#') or line.strip() == '':
            continue
        if header_end == 0 and re.search(r'[A-Za-z]', line):
            header_end = i
            break

# Try to extract column names from the first non-header line
col_line = lines[header_end].strip()
if ',' in col_line:
    sep = ','
elif '\t' in col_line:
    sep = '\t'
else:
    sep = None  # whitespace

# If the line looks like column names, use it
if re.search(r'[A-Za-z]', col_line):
    colnames = re.split(sep if sep else r'\s+', col_line)
    colnames = make_unique(colnames)
    skiprows = header_end + 1
else:
    colnames = None
    skiprows = header_end

# Read the data
if sep:
    df = pd.read_csv(filename, sep=sep, skiprows=skiprows, names=colnames)
else:
    df = pd.read_csv(filename, delim_whitespace=True, skiprows=skiprows, names=colnames)

# Clean up string columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip('"').str.strip()

# Preview
df.head()
df.to_csv(filename + '.csv', index=False)
files.download(filename + '.csv')


Saving Boundary_Fault to Boundary_Fault (1)
Saving Boundary_Fault_West to Boundary_Fault_West (1)
Saving Branched_Fault_NW to Branched_Fault_NW (1)
Saving Closing_Fault_East to Closing_Fault_East (1)
Saving Closing_Fault_South to Closing_Fault_South
Saving Closing_Fault_South_1 to Closing_Fault_South_1
Saving Closing_Fault_South_2 to Closing_Fault_South_2
Saving Closing_Fault_South_3 to Closing_Fault_South_3
Saving Closing_Fault_West_1 to Closing_Fault_West_1
Saving Closing_Fault_West_2 to Closing_Fault_West_2
Saving Closing_Fault_West_3 to Closing_Fault_West_3
Saving Main_Fault_East_1 to Main_Fault_East_1
Saving Main_Fault_East_2 to Main_Fault_East_2
Saving Main_Fault_NE to Main_Fault_NE
Saving Main_Fault_NS_1 to Main_Fault_NS_1
Saving Main_Fault_NS_2 to Main_Fault_NS_2
Saving Main_Fault_South to Main_Fault_South
Saving Main_Fault_West_1 to Main_Fault_West_1
Saving Main_Fault_West_2a to Main_Fault_West_2a
Saving Main_Fault_West_3a to Main_Fault_West_3a
Saving NNE-SSW to NNE-SSW
Saving

/tmp/ipython-input-3574265439.py:55: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(filename, delim_whitespace=True, skiprows=skiprows, names=colnames)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Converting Fault polygons files to csv format**

In [ ]:
# Colab: parse multiple Petrel ASCII / text files -> individual CSVs + one ZIP
!pip install -q --no-warn-script-location pandas

from google.colab import files
import pandas as pd
import re, zipfile, os
from io import StringIO

print("Select one or more Petrel ASCII / text files to upload (Ctrl/Cmd+click to multi-select)...")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No files uploaded.")

# Helper: detect sentinel values in header text like: NULL . -999.25 : NULL VALUE
def detect_sentinels(text, max_lines=200):
    pat = re.compile(r'NULL[^0-9\-\+]*([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)', re.IGNORECASE)
    sents = set()
    for i, line in enumerate(text.splitlines()):
        if i > max_lines: break
        m = pat.search(line)
        if m:
            try:
                sents.add(float(m.group(1)))
            except:
                pass
    return sorted(sents)

# Replace sentinels (exact numeric matches) with pandas NaN
def replace_sentinels_in_df(df, sentinels):
    if not sentinels:
        return df
    repl = {s: pd.NA for s in sentinels}
    repl.update({str(s): pd.NA for s in sentinels})
    df = df.replace(repl)
    # try numeric conversion where possible
    for c in df.columns:
        df[c] = pd.to_numeric(df[c].astype(str).str.replace(',', ''), errors='ignore')
    return df

# Robust parser for a single file (Petrel-like)
def parse_petrel_file(path):
    with open(path, 'r', errors='ignore') as f:
        lines = f.readlines()

    # Find first non-empty, non-comment line and any 'END HEADER'
    first_non_comment = None
    header_line_index = None
    data_start_index = None
    for i, line in enumerate(lines):
        s = line.strip()
        if s == "":
            continue
        if first_non_comment is None:
            first_non_comment = i
        if re.search(r'END\s+HEADER', line, re.IGNORECASE):
            data_start_index = i + 1
            break

    if data_start_index is None:
        if first_non_comment is None:
            raise ValueError("File appears empty (no data lines).")
        sample_line = lines[first_non_comment].strip()
        if re.search(r'[A-Za-z]', sample_line):
            header_line_index = first_non_comment
            data_start_index = first_non_comment + 1
        else:
            header_line_index = None
            data_start_index = first_non_comment

    inspect_index = header_line_index if header_line_index is not None else data_start_index
    if inspect_index is None or inspect_index >= len(lines):
        raise ValueError("Could not determine where data starts in file.")

    inspect_line = lines[inspect_index].strip()

    # Detect delimiter
    if ',' in inspect_line:
        sep = ','
    elif '\t' in inspect_line:
        sep = '\t'
    else:
        # prefer regex whitespace if there are runs of multiple spaces
        if re.search(r'\s{2,}', inspect_line):
            sep = r'\s+'
        else:
            sep = r'\s+'

    # Build text chunk for pandas
    if header_line_index is not None:
        text_chunk = ''.join(lines[header_line_index:])  # include header line
    else:
        text_chunk = ''.join(lines[data_start_index:])  # no header

    # Try reading with pandas
    try:
        if sep in (',', '\t'):
            if header_line_index is not None:
                df = pd.read_csv(StringIO(text_chunk), sep=sep, engine='python')
            else:
                df = pd.read_csv(StringIO(text_chunk), sep=sep, engine='python', header=None)
        else:
            if header_line_index is not None:
                df = pd.read_csv(StringIO(text_chunk), sep=sep, engine='python')
            else:
                df = pd.read_csv(StringIO(text_chunk), sep=sep, engine='python', header=None)
    except Exception as e:
        # Fallback: manual whitespace/token split per line
        rows = []
        for ln in text_chunk.splitlines():
            if not ln.strip():
                continue
            # try to protect commas inside quotes
            if '"' in ln or "'" in ln:
                # replace commas inside quotes with a token
                def _replace_inside_quotes(s, char=',', token='<<C>>'):
                    out = []
                    in_q = False; qchar = None
                    for ch in s:
                        if ch in ('"', "'"):
                            if not in_q:
                                in_q = True; qchar = ch
                            elif ch == qchar:
                                in_q = False; qchar = None
                            out.append(ch)
                        elif in_q and ch == char:
                            out.append(token)
                        else:
                            out.append(ch)
                    return ''.join(out)
                ln2 = _replace_inside_quotes(ln)
                parts = re.split(r'\s+', ln2.strip())
                parts = [p.replace('<<C>>', ',') for p in parts]
            else:
                parts = re.split(r'\s+', ln.strip())
            rows.append(parts)
        if not rows:
            raise ValueError("Fallback split produced no rows.")
        maxc = max(len(r) for r in rows)
        norm = [r + ['']*(maxc-len(r)) for r in rows]
        # if header_line_index was present, treat first row as header
        if header_line_index is not None:
            header = [h.strip() if h.strip() else f"col_{i+1}" for i,h in enumerate(norm[0])]
            data_rows = norm[1:]
        else:
            header = [f"col_{i+1}" for i in range(maxc)]
            data_rows = norm
        df = pd.DataFrame(data_rows, columns=header)

    # Strip surrounding quotes from string columns
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).str.strip().str.strip('"').str.strip("'")

    return df, sep, header_line_index is not None

# Common fallback sentinels if none are detected in header
COMMON_SENTINELS = [-999.25, -999.250000, -999.0, -999, -9999]

# Process multiple uploaded files
out_csv_paths = []
for fname in uploaded.keys():
    print("Processing:", fname)
    try:
        # Try to detect sentinel values from first ~40 lines
        try:
            with open(fname, 'r', errors='ignore') as fh:
                head_text = ''.join([next(fh) for _ in range(40)])
        except Exception:
            head_text = ""
        detected_sent = detect_sentinels(head_text)
        sentinels = detected_sent if detected_sent else COMMON_SENTINELS

        df, used_sep, had_header = parse_petrel_file(fname)
        df = replace_sentinels_in_df(df, sentinels)

        out_name = os.path.splitext(fname)[0] + "_converted.csv"
        df.to_csv(out_name, index=False)
        out_csv_paths.append(out_name)
        print(f" -> saved: {out_name}  (rows={len(df):,}, cols={df.shape[1]})")
    except Exception as e:
        print(f"ERROR parsing {fname}: {e}")

if not out_csv_paths:
    raise SystemExit("No CSVs were generated.")

# Prepare ZIP of all CSVs for easy download
zipname = "all_converted_csvs.zip"
with zipfile.ZipFile(zipname, "w", zipfile.ZIP_DEFLATED) as zf:
    for p in out_csv_paths:
        zf.write(p, arcname=os.path.basename(p))

print("\nCreated ZIP:", zipname)
files.download(zipname)

# OPTIONAL: also provide individual CSV downloads (uncomment if you want)
# for p in out_csv_paths:
#     files.download(p)

print("\nDone. Converted files:")
for p in out_csv_paths:
    print("  -", p)


Select one or more Petrel ASCII / text files to upload (Ctrl/Cmd+click to multi-select)...


Saving Etive to Etive
Saving Ness to Ness
Saving Tarbert to Tarbert
Processing: Etive
 -> saved: Etive_converted.csv  (rows=104, cols=4)
Processing: Ness
 -> saved: Ness_converted.csv  (rows=92, cols=4)
Processing: Tarbert
 -> saved: Tarbert_converted.csv  (rows=121, cols=4)

Created ZIP: all_converted_csvs.zip


/tmp/ipython-input-548963754.py:37: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c].astype(str).str.replace(',', ''), errors='ignore')
/tmp/ipython-input-548963754.py:37: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c].astype(str).str.replace(',', ''), errors='ignore')
/tmp/ipython-input-548963754.py:37: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c].astype(str).str.replace(',', ''), errors='ignore')
/tmp/ipython-input-548963754.py:37: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Done. Converted files:
  - Etive_converted.csv
  - Ness_converted.csv
  - Tarbert_converted.csv


**Converting Seismic interpretation time txt files to csv format**

In [ ]:
# Colab multi-file converter  TXT to Csv
!pip install -q pandas
from google.colab import files
import pandas as pd, re, zipfile, os
from io import StringIO

print("Select one or more TXT files to upload (Ctrl/Cmd+click)...")
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No files uploaded.")

def simple_detect_sep(text):
    if '\t' in text: return '\t'
    if ',' in text and text.count(',') >= text.count(' '): return ','
    return r'\s+'

def convert_bytes_to_df(b, fname):
    txt = b.decode('utf-8', errors='ignore')
    # drop initial comments
    lines = [ln for ln in txt.splitlines() if ln.strip() and not ln.strip().startswith(('#','//','~','%'))]
    if not lines:
        return pd.DataFrame()
    # header detection
    first = lines[0].strip()
    alpha = sum(bool(re.search('[A-Za-z]', tok)) for tok in re.split(r'\s+', first))
    nums  = sum(1 for tok in re.split(r'\s+', first) if re.match(r'^-?\d+(\.\d+)?$', tok))
    has_header = alpha > nums and alpha>0

    sample = '\n'.join(lines[:30])
    sep = simple_detect_sep(sample)

    if sep in (',','\t'):
        if has_header:
            df = pd.read_csv(StringIO('\n'.join(lines)), sep=sep, engine='python')
        else:
            df = pd.read_csv(StringIO('\n'.join(lines)), sep=sep, engine='python', header=None)
    else:
        # fallback to robust splitting line-by-line
        rows=[]
        for ln in lines:
            parts = re.split(r'\s+', ln.strip())
            rows.append(parts)
        maxc = max(len(r) for r in rows)
        rows = [r + ['']*(maxc-len(r)) for r in rows]
        if has_header:
            header = rows[0]
            data = rows[1:]
            df = pd.DataFrame(data, columns=header)
        else:
            df = pd.DataFrame(rows)
    df = df.replace([-999.25, -999.250000, -999, -999.0], pd.NA)
    return df

out_csvs = []
for fname, b in uploaded.items():
    print("Converting:", fname)
    df = convert_bytes_to_df(b, fname)
    outname = fname.rsplit('.',1)[0] + '_converted.csv'
    df.to_csv(outname, index=False)
    out_csvs.append(outname)
    print(" -> saved:", outname)

# zip them
zipname = "all_converted_csvs.zip"
with zipfile.ZipFile(zipname, 'w', zipfile.ZIP_DEFLATED) as zf:
    for p in out_csvs:
        zf.write(p)
print("Created:", zipname)
files.download(zipname)


Select one or more TXT files to upload (Ctrl/Cmd+click)...


Saving Base Cretaceous.txt to Base Cretaceous.txt
Saving Top Etive.txt to Top Etive.txt
Saving Top Ness.txt to Top Ness.txt
Saving Top Tarbert.txt to Top Tarbert.txt
Converting: Base Cretaceous.txt
 -> saved: Base Cretaceous_converted.csv
Converting: Top Etive.txt
 -> saved: Top Etive_converted.csv
Converting: Top Ness.txt
 -> saved: Top Ness_converted.csv
Converting: Top Tarbert.txt
 -> saved: Top Tarbert_converted.csv
Created: all_converted_csvs.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>